<a href="https://colab.research.google.com/github/DDan991/12/blob/main/logistic_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gdown
url = 'https://drive.google.com/uc?id=1U1ckFG-C1A7j5OzZ_PlF-durNt2HTy7T'
output = 'Iris.csv'
gdown.download(url, output, quiet = False)

Downloading...
From: https://drive.google.com/uc?id=1U1ckFG-C1A7j5OzZ_PlF-durNt2HTy7T
To: /content/Iris.csv
100%|██████████| 5.11k/5.11k [00:00<00:00, 10.6MB/s]


'Iris.csv'

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import random_split

In [5]:
data0 = pd.read_csv('Iris.csv')
print(data0.columns.tolist())
print(len(data0))

['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']
150


In [6]:
data0

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [7]:
Name0 = data0.Species.unique().tolist()
Name = sorted(Name0)
print(Name)
N = list(range(len(Name)))
normal_mapping = dict(zip(Name,N))
reverse_mapping = dict(zip(N,Name))
print(len(N))
print(normal_mapping)
print(reverse_mapping)

['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
3
{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
{0: 'Iris-setosa', 1: 'Iris-versicolor', 2: 'Iris-virginica'}


In [8]:
data0['Species'] = data0['Species'].map(normal_mapping)

In [23]:
data0
Name

['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

In [24]:
m = len(data0)
M = list(range(m))
random.seed(2024)
random.shuffle(M)

NameError: ignored

In [12]:
dataY = data0['Species']
dataX = data0.drop('Species', axis = 1)
dataX = dataX.drop('Id', axis = 1)

In [13]:
trainX, testX, trainY, testY = train_test_split(dataX, dataY, test_size = 0.25, random_state = 42)

In [ ]:
print(type(trainX))
print(type(trainY))

In [15]:
trainY = torch.from_numpy(np.array(trainY)).type(torch.LongTensor)
trainX = torch.from_numpy(np.array(trainX)).float()
testY = torch.from_numpy(np.array(testY)).type(torch.LongTensor)
testX = torch.from_numpy(np.array(testX)).float()

In [16]:
batch_size = 100
n_iters = 10000
num_epochs = n_iters/(len(trainX)/batch_size)
num_epochs = int(num_epochs)

In [17]:
num_epochs

8928

In [19]:
train_ds = torch.utils.data.TensorDataset(trainX, trainY)
test_ds = torch.utils.data.TensorDataset(testX, testY)

In [21]:
batch_size = 16
train_loader = DataLoader(train_ds, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
test_loader = DataLoader(test_ds, batch_size, num_workers = 4, pin_memory = True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [27]:
class LogisticRegressionModel(nn.Module):
  def __init__(self, input_dim, output_dim):
    super(LogisticRegressionModel, self).__init__()
    self.linear = nn.Linear(input_dim, output_dim)

  def forward(self,x):
    out = self.linear(x)
    return out

input_dim = 4
output_dim = len(Name)
model = LogisticRegressionModel(input_dim, output_dim)
error = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [28]:
count = 0
loss_list = []
iteration_list = []

for epoch in range(num_epochs):
  for i, (image, labels) in enumerate(train_loader):
    train = Variable(image.view(-1,4))
    labels = Variable(labels)
    optimizer.zero_grad()
    outputs = model(train)
    loss = error(outputs, labels)
    loss.backward()
    optimizer.step()
    count += 1

    if count % 50 == 0:
      correct = 0
      total = 0

      for images, labels in test_loader:
        test = images.view(-1,4)
        outputs = model(test)
        predicted = torch.max(outputs.data, 1)[1]
        total += len(labels)
        correct += (predicted == labels).sum()

      accuracy = 100* correct / float(total)
      loss_list.append(loss.data
                       )
      iteration_list.append(count)
    if count % 50 ==0:
      print('Iteration: {} Loss: {} Accuracy: {}%'.format(count, loss.data,accuracy))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Iteration: 50 Loss: 1.2734726667404175 Accuracy: 39.47368240356445%
Iteration: 100 Loss: 0.8742550611495972 Accuracy: 65.78947448730469%
Iteration: 150 Loss: 0.9247867465019226 Accuracy: 60.52631759643555%
Iteration: 200 Loss: 0.8655244708061218 Accuracy: 63.157894134521484%
Iteration: 250 Loss: 0.8166506886482239 Accuracy: 71.0526351928711%
Iteration: 300 Loss: 0.8385671973228455 Accuracy: 71.0526351928711%
Iteration: 350 Loss: 0.7835979461669922 Accuracy: 71.0526351928711%
Iteration: 400 Loss: 0.768263041973114 Accuracy: 71.0526351928711%
Iteration: 450 Loss: 0.8598615527153015 Accuracy: 71.0526351928711%
Iteration: 500 Loss: 0.7412436604499817 Accuracy: 71.0526351928711%
Iteration: 550 Loss: 0.7882394790649414 Accuracy: 71.0526351928711%
Iteration: 600 Loss: 0.747413158416748 Accuracy: 71.0526351928711%
Iteration: 650 Loss: 0.7735509276390076 Accuracy: 73.68421173095703%
Iteration: 700 Loss: 0.7352921366691589 Accuracy: 73.68421173095703%
Iteration: 750 Loss: 0.759626030921936 Accur

Exception ignored in: <function _after_fork at 0x7b5565205b40>
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1635, in _after_fork
    thread._reset_internal_locks(False)
  File "/usr/lib/python3.10/threading.py", line 885, in _reset_internal_locks
    def _reset_internal_locks(self, is_alive):
KeyboardInterrupt: 


KeyboardInterrupt: ignored

In [29]:
model

LogisticRegressionModel(
  (linear): Linear(in_features=4, out_features=3, bias=True)
)

In [30]:
test_outputs = model(testX)
test_predicted = torch.max(test_outputs.data,1)[1]

In [31]:
ANS = testY.numpy()
PRED = test_predicted.numpy()
accuracy = accuracy_score(ANS,PRED)
print(accuracy)

1.0
